#Understand and order data

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#LightFM is a Python implementation of a number of popular recommendation algorithms:

It also makes it possible to incorporate both item and user metadata into the traditional matrix factorization algorithms. It represents each user and item as the sum of the latent representations of their features, thus allowing recommendations to generalise to new items (via item features) and to new users (via user features).

In [2]:
# Install lightFM, takes around 15 seconds
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 5.0 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705378 sha256=34fdd0aea5275ffad5e87a74b7c0f4223d41c56a3a1f9290cd762ff1f77eb510
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


The first step is to get the Movielens data. This is a classic small recommender dataset, consisting of around 950 users, 1700 movies, and 100,000 ratings. The ratings are on a scale from 1 to 5, but we’ll all treat them as implicit positive feedback in this example.

Fortunately, this is one of the functions provided by LightFM itself.

In [3]:
# Import our modules
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM
# Use one of LightFM's inbuild datasets, setting the minimum rating to return at over 4.0
data = fetch_movielens(min_rating = 4.0)
data

{'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 5469 stored elements in COOrdinate format>,
 'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 49906 stored elements in COOrdinate format>}

In [4]:
# Get our key and value from our dataset
# By printing it, we see it's comprised of a data segments containing test, train, item_features, item_feature_labels & item_labels 
for key, value in data.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse.csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [5]:
# What type of data are we working with? coo_matrix
type(data['train'])

scipy.sparse.coo.coo_matrix

In [9]:
# Each row represents a user, and each column an item. 
# We use .tocsr() to view it as a Compressed Sparse Row format, it's an inbuilt function in the coo_matrix object
m1 = data['train'].tocsr()

print(m1[0,0])
print(m1[0,1])

  (0, 0)	5
  (0, 2)	4
  (0, 5)	5
  (0, 6)	4
  (0, 8)	5
  (0, 11)	5
  (0, 12)	5
  (0, 13)	5
  (0, 14)	5
  (0, 15)	5
  (0, 17)	4
  (0, 18)	5
  (0, 21)	4
  (0, 22)	4
  (0, 24)	4
  (0, 27)	4
  (0, 31)	5
  (0, 38)	4
  (0, 41)	5
  (0, 42)	4
  (0, 43)	5
  (0, 44)	5
  (0, 45)	4
  (0, 46)	4
  (0, 47)	5
  :	:
  (0, 223)	5
  (0, 226)	4
  (0, 227)	5
  (0, 228)	4
  (0, 229)	4
  (0, 233)	4
  (0, 234)	5
  (0, 235)	4
  (0, 237)	4
  (0, 238)	4
  (0, 240)	4
  (0, 241)	5
  (0, 245)	5
  (0, 247)	4
  (0, 248)	4
  (0, 249)	4
  (0, 250)	4
  (0, 252)	5
  (0, 255)	4
  (0, 256)	4
  (0, 257)	5
  (0, 266)	4
  (0, 267)	5
  (0, 268)	5
  (0, 269)	5
5
0


**coo_matrix - A sparse matrix in COOrdinate format - Intended Usage:**

- COO is a fast format for constructing sparse matrices
- Once a matrix has been constructed, convert to CSR or CSC format for fast arithmetic and matrix vector operations
- By default when converting to CSR or CSC format, duplicate (i,j) entries will be summed together.  This facilitates efficient construction of finite element matrices and the like. (see example)
<div>
<img src="https://imgs.developpaper.com/imgs/10502062C-0.gif" width="500"/>
</div>

In [10]:
print(repr(data['train'])) # rept() is used in debugging to get a string representation of object
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


# Let's now create and train our model

**Four loss functions are available:**

- **logistic**: useful when both positive (1) and negative (-1) interactions are present.
- **BPR**: Bayesian Personalised Ranking pairwise loss. Maximises the prediction difference between a positive example and a randomly chosen negative example. Useful when only positive interactions are present and optimising ROC AUC is desired.
- **WARP**: Weighted Approximate-Rank Pairwise loss. Maximises the rank of positive examples by repeatedly sampling negative examples until rank violating one is found. Useful when only positive interactions are present and optimising the top of the recommendation list (precision@k) is desired.
- **k-OS WARP**: k-th order statistic loss. A modification of WARP that uses the k-th positive example for any given user as a basis for pairwise updates.

**Two learning rate schedules are available:**
- adagrad
- adadelta

In [11]:
# Creat our model object from LightFM
# We specify the loss type to be WARP (Weighted Approximate-Rank Pairwise )
model = LightFM(loss = 'warp')

In [12]:
# Extract our training and test datasets
train = data['train']
test = data['test']

In [13]:
# Fit our model over 10 epochs
model.fit(train, epochs=10)

# Performance Evaluation

We use Precision and AUC to avaluate our model performance.

**The ROC AUC metric for a model**: the probability that a randomly chosen positive example has a higher score than a randomly chosen negative example.

**The precision at k metric for a model**: the fraction of known positives in the first k positions of the ranked list of results.

In [14]:
# Evaluate it's performance
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.48, test 0.08.
AUC: train 0.94, test 0.91.


# Let's see what movies are recommended for some users

In [15]:
# Function credit goes to Arun Mathew Kurian
# Let's test it out and see how well it works 
# https://towardsdatascience.com/how-to-build-a-movie-recommender-system-in-python-using-lightfm-8fa49d7cbe3b
def sample_recommendation(model, data, user_ids):
    '''uses model, data and a list of users ideas and outputs the recommended movies along with known positives for each user'''
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        
        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['item_labels'][np.argsort(-scores)]
      
        print("User %s" % user_id)
        print("Known positives:")
        
        # Print the first 3 known positives
        for x in known_positives[:3]:
            print("%s" % x)
        
        # Print the first 3 recommended movies
        print("Recommended:")
        for x in top_items[:3]:
            print("%s" % x)
        print("\n")

In [16]:
# Testing on users: 6, 125 and 336
sample_recommendation(model, data, [6, 125, 336])

User 6
Known positives:
Get Shorty (1995)
Twelve Monkeys (1995)
Babe (1995)
Recommended:
Raiders of the Lost Ark (1981)
Blade Runner (1982)
Empire Strikes Back, The (1980)


User 125
Known positives:
Jungle2Jungle (1997)
Kull the Conqueror (1997)
Scream (1996)
Recommended:
L.A. Confidential (1997)
Scream (1996)
Seven Years in Tibet (1997)


User 336
Known positives:
Mr. Holland's Opus (1995)
Star Wars (1977)
Ace Ventura: Pet Detective (1994)
Recommended:
Return of the Jedi (1983)
Star Wars (1977)
Independence Day (ID4) (1996)




### Learn to build and create your own datasets for LightFM here

https://lyst.github.io/lightfm/docs/examples/dataset.html